In [5]:
import pandas as pd
import folium
foley_df = pd.read_excel('Foley_Comp2_879.xlsx')
foley_df['Wetlands Coverage']


# visual chart attention grabber for stats slide
# cut the word count in half per slide
# important slides have two versions 
# work flow diagram (previous method) vs (new and improved)
# typo on hey i recognize the data slide
# add pointers on old pricing method slide (1000 rows)
# reduce words on alex and syd's slide, highlight takeaways 
# DBSCAN parameters
# DBSCAN not a circle, just a visual aid
# pick an example and go through the process (give one property's offer price)
# goal 1 : find where we wanna go (add to slides) 
# goal 2: calculate what we want to price it at (add to slides) 
# bumper stickers!
# SKLearn and DBSCAN
# prompt engineer, maps
# keep background short!

0      17.12
1      41.40
2      16.74
3        NaN
4        NaN
       ...  
305      NaN
306      NaN
307      NaN
308      NaN
309      NaN
Name: Wetlands Coverage, Length: 310, dtype: float64

In [15]:
foley_df.columns

Index(['APN', 'Lot Acres', 'Owner Name(s)', 'Mail Names', 'Owner 1 Full Name',
       'Owner 1 First Name', 'Owner 1 Middle Name', 'Owner 1 Last Name',
       'Mail Full Address', 'Mail City', 'Mail State', 'Mail Zip',
       'Parcel Full Address', 'Parcel City', 'Parcel County', 'Latitude',
       'Longitude', 'Hyperlink', 'Calc Acreage', 'Land Use', 'Land Locked',
       'Road Frontage', 'Wetlands Coverage', 'FEMA Flood Coverage',
       'Slope AVG', 'Buildability total (%)', 'Buildability area (acres)',
       'Flat slope (0-.5%)', 'Minimal Slope (.5-5%)', 'Moderate Slope (5-10%)',
       'Heavy Slope (10-15%)', 'Extreme Slope (15%+)', 'Elevation AVG',
       'Elevation Min', 'Elevation Max', 'Slope Min', 'Slope Max',
       'Total Assessed Value', 'Land Assessed Value',
       'Improvement Assessed Value', 'Total Market Value', 'Land Market Value',
       'Current Sale Price', 'FL Inside SFHA', 'FL FEMA Flood Zone',
       'FL FEMA Map Date', 'TLP Estimate'],
      dtype='object')

In [22]:
foley_df['Wetlands Coverage'].unique() # 0 - 10 full price, 10 - 30: 80% , 30-50: 70%, 50+: 50%

array([1.712e+01, 4.140e+01, 1.674e+01,       nan, 1.322e+01, 2.926e+01,
       2.419e+01, 1.147e+01, 1.743e+01, 4.520e+01, 2.200e+00, 5.170e+00,
       9.691e+01, 3.176e+01, 8.720e+00, 1.325e+01, 1.628e+01, 3.314e+01,
       9.940e+00, 3.506e+01, 7.100e+00, 3.174e+01, 4.499e+01, 2.443e+01,
       2.000e-01, 4.100e+00, 7.610e+00, 1.000e+02, 1.340e+00, 6.890e+00,
       4.600e-01, 2.940e+00, 1.354e+01, 4.160e+00, 4.853e+01, 7.710e+00,
       9.837e+01, 5.300e+00, 6.780e+00, 2.750e+00, 2.470e+00, 9.980e+00,
       2.011e+01, 2.670e+00, 6.677e+01, 1.300e+00, 3.889e+01, 6.130e+00,
       4.210e+00, 2.533e+01, 1.100e-01, 1.080e+00, 1.792e+01, 9.008e+01,
       4.390e+00, 5.803e+01, 9.921e+01, 2.030e+01, 1.108e+01, 2.416e+01,
       2.120e+00, 4.707e+01, 2.755e+01, 9.100e-01, 2.604e+01, 5.100e-01,
       7.120e+00, 7.779e+01, 3.000e-02, 8.364e+01, 7.570e+00, 1.100e+00,
       2.439e+01, 2.610e+00, 2.250e+00, 8.100e-01, 8.970e+00, 2.555e+01,
       7.760e+00, 2.449e+01, 1.508e+01, 1.960e+00, 

In [100]:
#convert to float
cols = ['Wetlands Coverage', 'Extreme Slope (15%+)', 'Road Frontage', 'Lot Acres']

foley_df[cols] = (
    foley_df[cols]
    .replace(',', '', regex=True)      # remove commas
    .apply(pd.to_numeric)  # convert to float
)


In [101]:
import pandas as pd
import numpy as np

def weight(df):
    # Wetlands Coverage weight (0–100 range)
    bins = [0, 10, 30, 50, 100]        # boundaries
    labels = [1, 0.8, 0.7, 0.5]        # weights for each interval
    wet_weight = pd.cut(df['Wetlands Coverage'], bins=bins, labels=labels, right=False).astype(float)
    # right=False means intervals are [start, end) so 10 belongs to the next bin

    # Extreme slope weight
    slope_weight = np.where(df['Extreme Slope (15%+)'] > 50, 0.7, 1)
    # np.where(condition, value_if_true, value_if_false)

    # Road weight
    road_weight = np.where((df['Road Frontage'] > 500) & (df['Lot Acres'] > 3), 1.05, 1)

    # Final combined weight
    return wet_weight * slope_weight * road_weight


In [99]:
# Average price per acre
avg_price_per_acre = 52124.0

# Create filtered DataFrame
filtered_df = foley_df.dropna(
    subset=['Wetlands Coverage', 'Extreme Slope (15%+)', 'Road Frontage']
).copy()

# Calculate Suggested and Adjusted Price
suggested_price = (filtered_df["Lot Acres"] * avg_price_per_acre).round().astype(int)
adjusted_price =  suggested_price * weight(filtered_df)

filtered_df.insert(2, "Suggested Price", suggested_price)
filtered_df.insert(3, "Adjusted Price", adjusted_price)

display_df = filtered_df[
    ['Lot Acres','Wetlands Coverage', 'Extreme Slope (15%+)', 'Road Frontage',
     'Suggested Price', 'Adjusted Price']
].copy()

# Preview
display_df

,Lot Acres,Wetlands Coverage,Extreme Slope (15%+),Road Frontage,Suggested Price,Adjusted Price
0,4.800,17.12,0.05,380.35,250195,200156.0
1,3.600,41.40,1.53,347.98,187646,131352.2
2,2.596,16.74,1.72,200.53,135314,108251.2
6,3.260,13.22,19.75,299.14,169924,135939.2
9,3.800,29.26,8.12,50.00,198071,158456.8
...,...,...,...,...,...,...
291,4.200,31.24,0.76,244.68,218921,153244.7
292,2.400,0.02,6.93,1103.12,125098,125098.0
293,4.400,24.02,0.66,489.31,229346,183476.8
295,2.578,14.26,20.57,733.43,134376,107500.8
